In [151]:
import os
import json
from dotenv import load_dotenv
from googleapiclient.discovery import build

load_dotenv()

True

In [152]:
DEVELOPER_KEY = os.environ.get('YT_KEY')
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

In [153]:
yt = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)

In [154]:
# Individuo le finestre temporali per le ricerche, con i rispettivi parametri
windows = [
    {"label": "approdo_maldini", "query": "Maldini Milan 2019 dirigenza", "after": "2019-05-20T00:00:00Z", "before": "2019-07-01T00:00:00Z"},
    {"label": "scudetto", "query": "Milan campione 2022 Maldini", "after": "2022-05-15T00:00:00Z", "before": "2022-06-15T00:00:00Z"},
    {"label": "licenziamento", "query": "Maldini licenziato Cardinale", "after": "2023-06-01T00:00:00Z", "before": "2023-06-30T00:00:00Z"},
    {"label": "cardinale_out", "query": "#CardinaleOut Milan", "after": "2023-06-01T00:00:00Z", "before": "2023-12-31T00:00:00Z"}
]

In [155]:
# Individuo i canali più influenti da cui estrarre i video
def get_channels(query, max_results):
    channels = set()

    request = yt.search().list(
        part='snippet',
        q=query,
        type='channel',
        maxResults=max_results
    )

    try:
        response = request.execute()

        for item in response.get('items', []):
            channel_name = item['snippet']['title']
            channel_id = item['id']['channelId']
            channels.add((channel_id, channel_name))

    except Exception as e:
        print(f"Error fetching channels: {e}")

    return channels

In [156]:
channels = get_channels("AC Milan", 15)
for channel in channels:
    print(channel[1])

len(channels)

AC Milan Addicted
Notizie di AC Milan Ufficiale
Ac Milan Calcio
AC Milan
AcmDevil
La Storia del Milan
Rinaldo Morelli
SempreMilan
AC Milan Academy Junior Camp - Sporteventi
Ac Milan 2015
Football Kush
Lorenzo Lollo - I Milanisti Official
MILAN HELLO - ANDREA LONGONI
La Taberna del Milan
Milan Actu


15